# Homework 7

## CSCI E-82A  

[**Value at Risk (VaR)**](https://en.wikipedia.org/wiki/Value_at_risk) is a commonly used measure of financial risk. VaR is as an **order statistic**, being defined at a probability quantile $p$. For example, for the 1% var is there is a probability of $0.01$ that the loss will be at the lower 1% quantile of the returns or greater.      

Returns of financial assets are well know to be non-Gaussian. This presents significant problems for analysis, including risk analysis. Many approaches have been used to create probability models which better represent the returns of financial assets.   

One possibility is to use Gaussian mixture models to model the distribution of financial returns. In this assignment, you will model returns of the [S&P 500 Index](https://en.wikipedia.org/wiki/S%26P_500_Index) using Gaussian mixture models and then compute the VaR.    

> **Note:** In this exercise you will work with daily returns and therefore daily VaR. You may not be familiar with the concept of returns. The return is the difference between the value from one period (day) to the next (day). A **positive return** means the asset has increased in value. Whereas, a **negative return** means the asset has decreased in value, and represent losses. It is these losses the VaR model seeks to quantify.     

> More specifically, in this assignment you will work with log returns. Here the log returns are the returns of the log values of the series.         

## Load and Examine the Dataset

As a first step you will load a dataset containing approximately 5 years of daily index value data for the S&P 500 index (benchmark indicies do no have prices). 

Execute the code in the cell below to load the packages you will need for this assignment. 

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as nr
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline

Execute the code in the cell below to load the S&P 500 daily values, downloaded from Yahoo finance on October 8, 2019. 

In [ ]:
SP_Raw = pd.read_csv('GSPC_Yahoo_Oct_8_19.csv')

Execute the code in the cell below and examine the column names.

In [ ]:
SP_Raw.columns

There are a number of columns in this dataset. You will be working with the `Adj Close` column. The **adjusted close** is a final price of the day, adjusted for stock splits and dividend payments. 

Execute the code in the cell below to add a count of days from the start of the series and plot the time series of adjusted returns.

In [ ]:
SP_Raw['Day_Count'] = [float(i) for i in range(SP_Raw.shape[0])]
plt.plot(SP_Raw['Day_Count'],SP_Raw['Adj Close'])

There are several points to notice in this time series:   

1. There is a long-term trend of the daily values. We will remove this effect so we can examine the day-to-day returns (changes) without being influenced by the overall trend.
2. There is considerable volatility (variance) in this time series. We want to calculated the VaR of this time series, and in detail this variability will affect the results. We will ignore this problem for this assignment.

## Remove the Trend

To compute returns the trend must be removed from the time series. In this case, we will simply compute the remove the linear trend.   
 
 As a first step you must compute two numpy arrays:
 - The feature (independent variable) is the 'Day_Count' column.
 - The label (dependent variable) is the log of the 'Adj Close' column.     
 
Make sure to apply the [numpy ravel](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ravel.html) method to each array to ensure the dimensionality is correct.  
 
> **Note:** It is standard practice in capital markets analytics to work with log returns. The intuition for this is that the price of capital market assets cannot drop below 0, so taking the logarithm transforms the return distribution to be closer to Gaussian. Often the word *return* actually means *log return*. In the rest of this document, we will mostly just use the term *return*, but will mean *log return* unless otherwise noted. 

In the cell below do the following:  
- Scale the feature column with the scale function from sklearn.preprocessing. 
- Define and fit the model using the LinearRegression model and fit method from sklearn.linear_model. you will need to reshape the feature column as, [`.reshape(-1,1)`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html), since there is only one column in the feature array. 
- Subtract the predicted value from the model (use the `predict` method) to find a single column array. You will need to apply `.reshape(number_of_days, 1)` to this result to get the dimensions correct. 
- [Concatenate ](https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html)the following as columns of a numpy array; 
  - The unscaled label values. 
  - The scaled label values.
  - The difference between the log values and predicted values from the regression. 
  
You will need to ensure the dimensions of all these columns are correct before applying the `concatenate` method. 

Now, make a plot of the difference between the log values and predicted values. Ensure that the trend removal process has produced a reasonable result.

## Compute the Returns

With the trend removed from the daily values, its time to compute the returns. In the cell below do the following:

- Compute the returns using the numpy [`diff`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.diff.html) method. 
- The length of the difference (return) series will be 1 less than the original series. You will need to add a value of 0.0 to the start of the series, using the numpy `concatenate` method. 
- Concatenate the difference value column to the numpy array you created previously. Remember to apply `ravel` and `reshape` to the difference array. 

In the cell below, plot the time series of the returns.

Examine this series. Notice that the volatility (local variance) of these returns change with time. Volatility remains relatively constant for a period, and then changes. This is known as the [clustering of volatility effect](https://en.wikipedia.org/wiki/Volatility_clustering).  To model volatility clustering, Prof Robert Engel to develop the [ARCH model](http://www.econ.uiuc.edu/~econ508/Papers/engle82.pdf) which won him the Nobel prize in economics. 

In this exercise we will ignore the non-stationarity (clustering of volatility) nature of the return series. This may lead to some less than idea results.  

## Exploration of the Return Series

Now you will explore the probability properties of the return series and compute the empirical VaR. You will then compare this to the VaR computed using a single Gaussian distribution. 

In the cell below, use the scale function from sklearn.preprocessing to scale the return series. 

Now, plot a histogram of the return values, using 50 bins. 

**Question:** Carefully examine this histogram. In particular, look for signs of asymmetry and the 'weight' of the tails. What do you observe in terms of differences of this empirical distribution compared to a textbook Gaussian (Normal) distribution. 

> **Hint:**: There are two well know reasons that Gaussian (Normal) distributions do not model the (log) returns of financial assets. The actual returns are asymmetric and have greater 'weight' in the tails than would be modeled with a Gaussian distribution. 

ANS: 

Now, you will compute the 1% (p = 0.01) **empirical VaR** using the values of the return series. Write a function following this recommended process:
1. Sort the values with the numpy sort method.
2. Find the index of the at the 0.01 quantile.
3. Print the value for the index.

Execute your function and examine the results.

> **Note:** *Emperical VaR* is the VaR computed using the log return series. The computation is simply a matter of counting the ordered returns from the lowest value to the $p * series\ length$ value. 

Compare the empirical VaR you computed above to the **Normal VaR**. Since the return series has already been standardized you can compute the Normal 1% VaR using the [norm.ppf](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) function from scipy.stats, with $q = 1.0$.  

> **Note:** The Normal or Gaussian VaR is simply the pth quantile of the Normal distribution, given the estimated variance of the 0 mean return series. In this case, the return series is standardized so, the variance is just $\sigma^2 = 0.01$.

Compare the empirical 1% VaR with the Normal 1% VaR. Which VaR measure estimates higher losses for the worst 1% of trading days? Which VaR measure do you think is more realistic and trustworthy?  

ANS: 

## The Gaussian MIxture Model   

Now you are ready to model the return series as a GMM and then examine the properties of the of returns simulated from this model. In this case you will compute the GMM model using maximum likelihood Estimation (MLE) EM algorithm with the [GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) method from the sklearn.mixture package. 

In the cell below do the following:
1. (Optionally) Create a function to print attributes of you GMM model; .weights_, .means_, .covariances_. Such a function will help you understand the components of your model. 
2. Set a numpy.random seed of 654.
3. Define a GaussianMixture model with the following arguments:
  - n_components = 3, to fit 3 Gaussian distributions in your mixture,
  - verbose = 2, to see some information on the progress of the EM algorithm,
  - covariance_type = 'spherical', since there is only a variance for each univariate distribution,
  - reg_covar = 0.1, to ensure proper regularization. 
4. Fit the log return series with the .fit method. You will need to apply `.reshape(-1, 1)` so that the dimensionality of the array is correct. 
5. Print the results of your model using the function you have defined. 

Examine the weights, means and covariances (actually the univariate variance) for the model you have created. Given these parameters, what does this tell you about the major (dominant) Gaussian vs. the other components of this model? 

ANS: 

With your GMM model computed, you will now simulate returns from the mixture distribution and compute the 1% VaR from the simulated values. In the cell below create code to perform the following operations for 10,000 samples: 
1. Create a function to perform the simulation with the following steps:
  - Start with an [empty](https://docs.scipy.org/doc/numpy/reference/generated/numpy.empty.html) numpy array of dimension (0). 
  - For each model weight, compute the fraction of the total realizations which use the parameters for that component of the mixture. It may help you when debugging to print this faction. 
  - For the fraction of realizations use the [normal](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.normal.html) to compute realizations of a univariate Normal (Gaussian) for each component. 
  - Concatenate the realizations computed from each component. 
2. Execute your function to compute the simulated return values. 
3. Compute and print the 1% VaR. 

Next, plot the histogram of your simulated returns with 200 bins. 

Compare this histogram to the histogram of the empirical returns. Paying attention to the downside tail, and keeping in mind that loss events can occur that did not actually occur in the history (empirical data), what is the key difference and do you think this difference might matter?

ANS: 

Keeping the differences of the two histogrames in mind, compare the 1% VaR you have computed from your GMM with the 1% empirical VaR you computed earlier. How big is this difference, and do you think it might matter and why?

ANS: 

## Bayesian Gaussian Mixture Model

As a final step you will model the return series as a Bayesian GMM and then examine the properties of the of returns simulated from this model. This model uses the Bayesian variational EM algorithm. In this case you will compute the Bayesian GMM model using maximum likelihood Estimation (MLE) EM algorithm with the [BayesianGaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.BayesianGaussianMixture.html#sklearn.mixture.BayesianGaussianMixture) method from the sklearn.mixture package. 

In the cell below do the following:
1. Set a numpy.random seed of 345.
3. Define a BayesianGaussianMixture model with the following arguments:
  - n_components = 3, to fit 3 Gaussian distributions in your mixture,
  - verbose = 2, to see some information on the progress of the EM algorithm,
  - covariance_type = 'spherical', since there is only a variance for each univariate distribution,
  - weight_concentration_prior = 0.1, as a prior on the component weights, which puts less mass on the center component of the mixture,
  - mean_prior = [0.0], as a prior on the component means, consistent with expected returns typically being close to 0,
  - covariance_prior = 1.0, as the variance prior for the standardized data,
  - max_iter = 500, since convergence seems to be a problem with this model. 
4. Fit the log return series with the .fit method. You will need to apply `.reshape(-1, 1)` so that the dimensionality of the array is correct. 
5. Print the results of your model using the function you have defined previously. 

Notice that convergence is slow and that one of the components has both small weight and unrealistically small variance. These facts seem to point to the model being over parameterized. 

Perhaps a model with 2 components, and fewer parameters, will work better? In the cell below, compute a Bayesian GMM with 2 mixture components, starting with a numpy.random.seed of 567. 

Notice that the weights of the 2 components are nearly equal and the means are both close to 0. However, the variances of the two components are quite different. 

To investigate this model further, in the cell below create and execute code to do the following:
- Simulate 10,000 realizations of returns from the 2 component Gaussian GMM. 
- Plot the histogram of these realizations using 200 bins. 
- Compute and print the 1% VaR using these realizations. 

Compare this histogram to the histogram of the empirical returns. Do you think this histogram represents a realistic model and why? 

ANS:  

Given your observations above, do you think the 1% VaR for this model is likely to be a reasonable representation of risk and why?

ANS: 